In [1]:
import pandas as pd
import time
from cricsheet.fow_analysis.collapses.extract_collapses import return_collapses

## Load and Format Single Match

In [72]:
filepath = '../data/raw/csv/howstat/fall_of_wickets/'
file = 'fow_1999.csv'
df = pd.read_csv(filepath+file, index_col=0, parse_dates=[2], infer_datetime_format=True)

In [73]:
df.groupby(['MatchId','MatchInnings', 'Team']).apply(return_collapses).reset_index()

,MatchId,MatchInnings,Team,level_3,start,end,runs,wickets_lost,batters
0,1999,1,England,0,5.0,7.0,0.0,"[5, 6, 7]","[Cook, Prior, Broad]"
1,1999,2,Australia,0,6.0,9.0,22.0,"[6, 7, 8, 9]","[Haddin, Hussey, Johnson, Siddle]"
2,1999,2,Australia,1,7.0,10.0,23.0,"[7, 8, 9, 10]","[Hussey, Johnson, Siddle, Doherty]"
3,1999,2,Australia,2,1.0,3.0,22.0,"[1, 2, 3]","[Watson, Ponting, Katich]"


Questions to answer:
- Number of collapses by Team, by year (unique collapses, innings with a collapse)
- Positions most often involved
- Batters most often involved

## Load and Format All Matches

### Experiments - Concatenating Dfs

~2500 matches. 
1) What is the most efficient way to load all dataframes?

2) Is it more efficient to load and format one-by-one, or concatenate into a single df and groupby the whole df


In [5]:
# Try with 100 dataframes initially

n = 100
filepath = '../data/raw/csv/howstat/fall_of_wickets/'

#### Method 1: Load all using glob generator + concat

In [50]:
import glob
import os

In [42]:
start = time.time()
all_files = glob.glob(os.path.join(filepath, "*.csv"))
all_files_to_load = all_files[:]

df_from_each_file = (pd.read_csv(f, index_col=0, parse_dates=[2], infer_datetime_format=True) for f in all_files_to_load)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

6.191442012786865 seconds


In [48]:
"""
Attempts:
13 seconds
6.19 seconds

"""

'\nAttempts:\n13 seconds\n6.19 seconds\n\n'

#### Method 2: os.listdir + concat

In [45]:
from os import listdir

start = time.time()
df = pd.concat([pd.read_csv(f'{filepath}/{f}', index_col=0, parse_dates=[2], infer_datetime_format=True) for f in os.listdir(filepath) if f.endswith('.csv')])

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

6.0129618644714355 seconds


In [46]:
"""
Attempts:
6.37 seconds
6.012
"""

'\nAttempts:\n6.37 seconds\n6.012\n'

#### Method 3: Dask

In [58]:
import dask.dataframe as dd

start = time.time()
df = dd.read_csv(f'{filepath}*.csv')

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

2.355700731277466 seconds


In [59]:
start = time.time()
df = df.compute()

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

30.62992286682129 seconds


In [60]:
"""
Attempts:
33 seconds
"""

'\nAttempts:\n33 seconds\n'

In [56]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, Unnamed: 0 to Player
dtypes: object(4), int64(5)

I prefer Method 1

### Apply Chosen Method

In [2]:
import glob
import os

filepath_fow = '../data/raw/csv/howstat/fall_of_wickets/'

all_fow = glob.glob(os.path.join(filepath_fow, "*.csv"))
all_fow_to_load = all_fow[:]

df_fow_from_each_file = (pd.read_csv(f, index_col=0, parse_dates=[2], infer_datetime_format=True) for f in all_fow_to_load)
concatenated_df_fow   = pd.concat(df_fow_from_each_file, ignore_index=True)

In [3]:
concatenated_df_fow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74100 entries, 0 to 74099
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   MatchId       74100 non-null  int64         
 1   MatchDate     74100 non-null  datetime64[ns]
 2   MatchInnings  74100 non-null  int64         
 3   Team          74100 non-null  object        
 4   TeamInnings   74100 non-null  object        
 5   Wicket        74100 non-null  int64         
 6   Runs          74100 non-null  int64         
 7   Player        74100 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 4.5+ MB


In [5]:
#concatenated_df_fow.to_csv('../data/interim/howstat/fall_of_wickets/fow_all.csv')
concatenated_df_fow = pd.read_csv('../data/interim/howstat/fall_of_wickets/fow_all.csv')

In [5]:
filepath_scores = '../data/raw/csv/howstat/scorecards/'

all_scores = glob.glob(os.path.join(filepath_scores, "*.csv"))
all_scores_to_load = all_scores[:]

df_scores_from_each_file = (pd.read_csv(f, index_col=0, parse_dates=[2], infer_datetime_format=True) for f in all_scores_to_load)
concatenated_df_scores   = pd.concat(df_scores_from_each_file, ignore_index=True)

In [6]:
concatenated_df_scores = concatenated_df_scores[['MatchId', 'MatchInnings', 'Team', 'TeamInnings', 'Player', 'R', 'BF']]
concatenated_df_scores['BattingPosition'] = concatenated_df_scores.groupby(['MatchId','MatchInnings', 'Team']).cumcount() + 1

In [7]:
concatenated_df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113386 entries, 0 to 113385
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MatchId          113386 non-null  int64  
 1   MatchInnings     113386 non-null  int64  
 2   Team             113386 non-null  object 
 3   TeamInnings      113386 non-null  object 
 4   Player           113386 non-null  object 
 5   R                113386 non-null  float64
 6   BF               113386 non-null  float64
 7   BattingPosition  113386 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 6.9+ MB


In [6]:
#concatenated_df_scores.to_csv('../data/interim/howstat/scorecards/scorecards_all.csv')
concatenated_df_scores = pd.read_csv('../data/interim/howstat/scorecards/scorecards_all.csv')

### Experiments - Fuzzy Matching


We want to get some information from the scorecards into the Fall of Wickets objects. Unfortunately the batter names don't match exactly (scorecards have initials as well). We need to do some fuzzy matching before joining info from the scorecards.

In [52]:
df_fow_to_merge = concatenated_df_fow[concatenated_df_fow.MatchId<=10]
df_scores_to_merge = concatenated_df_scores[concatenated_df_scores.MatchId<=10]

#### Method 1: fuzzy wuzzy

In [13]:
from cricsheet.utils import fuzzy_merge

start = time.time()

df_merged = fuzzy_merge(df_fow_to_merge, df_scores_to_merge, 'Player', 'Player', 80)

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

185.5373706817627 seconds


c:\users\adam\documents\pycharmprojects\cricsheet\cricsheet\src\cricsheet\utils.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
c:\users\adam\documents\pycharmprojects\cricsheet\cricsheet\src\cricsheet\utils.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


2 seconds for 10 matches.

183 seconds for 100 matches. 

We have 2400 so this won't scale. Estimate: >2hrs to match all.

#### Method 2: fuzzy-pandas

In [23]:
import fuzzy_pandas as fpd

start = time.time()

matches = fpd.fuzzy_merge(df_fow_to_merge, df_scores_to_merge,
                          left_on=['Player'],
                          right_on=['Player'], 
                          method='levenshtein',
                          ignore_case=True,
                          keep='match', 
                          join='left-outer', 
                          threshold=0.3)

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

51.402904748916626 seconds


15 seconds for 10 matches.

51 seconds for 20 matches. 

We have 2400 so this won't scale. Estimate: >2hrs to match all.

#### Method 3: difflib

In [53]:
import difflib
from functools import partial

start = time.time()

f = partial(
    difflib.get_close_matches, possibilities=df_scores_to_merge['Player'].tolist(), n=1, cutoff=0.3)

matches = df_fow_to_merge['Player'].map(f).str[0].fillna('')
scores = [
    difflib.SequenceMatcher(None, x, y).ratio() 
    for x, y in zip(matches, df_fow_to_merge['Player'])
]

df_fuzzy_matched = df_fow_to_merge.assign(best=matches, score=scores)

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

1.6973652839660645 seconds


0.85 seconds for 10 matches.

64 seconds for 100 matches.

548 seconds for 300 matches.

In [56]:
df_fuzzy_matched['score'].describe()

count    335.000000
mean       0.794888
std        0.065454
min        0.615385
25%        0.750000
50%        0.800000
75%        0.857143
max        0.909091
Name: score, dtype: float64

All of the above approaches match all of df_fow with all of df_scorecard.
That isn't necessary. We can do it for each Match, since the batters will match for each match.

### Load and Fuzzy-match a Match at a time

In [47]:
import glob
import os

filepath_scores = '../data/raw/csv/howstat/scorecards/'

all_scores = glob.glob(os.path.join(filepath_scores, "*.csv"))
all_scores_to_load = all_scores[:100]


filepath_fow = '../data/raw/csv/howstat/fall_of_wickets/'

all_fow = glob.glob(os.path.join(filepath_fow, "*.csv"))
all_fow_to_load = all_fow[:100]


In [6]:
import difflib
from functools import partial

def fuzzy_match(df1, df2, left_on, right_on):
    
    f = partial(
        difflib.get_close_matches, possibilities=df2[right_on].tolist(), n=2, cutoff=0.3)

    matches = df1[left_on].map(f).str[0].fillna('')
    scores = [
        difflib.SequenceMatcher(None, x, y).ratio() 
        for x, y in zip(matches, df1[left_on])
    ]

    df_fuzzy_matched = df1.assign(best=matches, score=scores)
    
    return df_fuzzy_matched

In [49]:
# check the match_ids match
l_merged_df = []

# for each fow file:
for i in range(len(all_fow_to_load)):
    #print(f'file: {i}')
    
    # read fow, read scorecard
    df_fow = pd.read_csv(all_fow_to_load[i], index_col=0, parse_dates=[2], infer_datetime_format=True)
    df_scores = pd.read_csv(all_scores_to_load[i], index_col=0, parse_dates=[2], infer_datetime_format=True)

    # select cols in scorecard, rename
    df_scores = df_scores[['MatchId', 'MatchInnings', 'Team', 'TeamInnings', 'Player', 'R', 'BF']]
    df_scores['BattingPosition'] = df_scores.groupby(['MatchId','MatchInnings', 'Team']).cumcount() + 1
    
    l_innings = []
    for innings in df_fow.MatchInnings.unique():
        #print(f'innings: {innings}')
        df_fow_innings = df_fow[df_fow.MatchInnings==innings]
        df_scores_innings = df_scores[df_scores.MatchInnings==innings]
        
        # fuzzy match on Player
        df_matched_innings = fuzzy_match(df_fow_innings, df_scores_innings, 'Player', 'Player')

        # merge cols from scores
        df_merged_innings = df_matched_innings.merge(df_scores_innings, 
                                  how='left', 
                                  left_on=['MatchId', 'MatchInnings', 'Team', 'TeamInnings', 'best'], 
                                  right_on=['MatchId', 'MatchInnings', 'Team', 'TeamInnings', 'Player']
                                 )

        # reformat
        df_merged_innings.drop(['Player_x', 'Player_y'], axis=1, inplace=True)
        df_merged_innings = df_merged_innings.rename({'best':'Player'}, axis=1)
        df_merged_innings['Player'] = df_merged_innings['Player'].apply(lambda x: re.sub('[!,*)@#%(&$_?.^†]', '', x))

        l_innings.append(df_merged_innings)        
    
    df_merged_match = pd.concat(l_innings)
    
    l_merged_df.append(df_merged_match)
    


In [51]:
df = pd.concat(l_merged_df)

In [8]:
from cricsheet.fow_analysis.collapses.preprocess_data import load_all_and_process

In [9]:
df = load_all_and_process()

In [10]:
df

,MatchId,MatchDate,MatchInnings,Team,TeamInnings,Wicket,Runs,Player,score,R,BF,BattingPosition
0,1,1877-03-15,1,Australia,1st,1,2,N Thompson,0.888889,1.0,0.0,2
1,1,1877-03-15,1,Australia,1st,2,40,T P Horan,0.714286,12.0,0.0,3
2,1,1877-03-15,1,Australia,1st,3,41,E J Gregory,0.777778,0.0,0.0,7
3,1,1877-03-15,1,Australia,1st,4,118,B B Cooper,0.750000,15.0,0.0,5
4,1,1877-03-15,1,Australia,1st,5,142,W E Midwinter,0.818182,5.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2,1088,1987-12-11,3,New Zealand,2nd,3,77,M D Crowe,1.000000,8.0,34.0,4
3,1088,1987-12-11,3,New Zealand,2nd,4,139,A H Jones,0.714286,64.0,137.0,3
4,1088,1987-12-11,3,New Zealand,2nd,5,153,D N Patel,0.714286,40.0,113.0,5
5,1088,1987-12-11,3,New Zealand,2nd,6,170,E J Gray,0.666667,14.0,68.0,6


In [5]:
start = time.time()

df_grouped = df.groupby(['MatchId','MatchInnings', 'Team']).apply(return_collapses).reset_index()

end = time.time()
time_taken = end - start
print(f'{time_taken} seconds')

1.5598597526550293 seconds


In [76]:
df_grouped.to_csv('../data/processed/collapses/all_collapses.csv')